In [1]:
import os

In [2]:
from git import Repo
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain

In [3]:

%pwd

'c:\\GenerativeAI\\Source-Code-Analysis-Using-GenAI\\research'

In [4]:

!mkdir test_repo

In [5]:
repo_path = "test_repo/"

Repo.clone_from("https://github.com/entbappy/End-to-end-ML-Project-Implementation", to_path=repo_path)

<git.repo.base.Repo 'c:\\GenerativeAI\\Source-Code-Analysis-Using-GenAI\\research\\test_repo\\.git'>

In [6]:
repo_path = "test_repo/"

loader = GenericLoader.from_filesystem(repo_path+'/src/mlProject',
                                        glob = "**/*",
                                       suffixes=[".py"],
                                       parser = LanguageParser(language=Language.PYTHON, parser_threshold=500)
)

In [7]:

documents = loader.load()

In [8]:
documents

[Document(page_content='import os\nimport sys\nimport logging\n\nlogging_str = "[%(asctime)s: %(levelname)s: %(module)s: %(message)s]"\n\nlog_dir = "logs"\nlog_filepath = os.path.join(log_dir,"running_logs.log")\nos.makedirs(log_dir, exist_ok=True)\n\n\nlogging.basicConfig(\n    level= logging.INFO,\n    format= logging_str,\n\n    handlers=[\n        logging.FileHandler(log_filepath),\n        logging.StreamHandler(sys.stdout)\n    ]\n)\n\nlogger = logging.getLogger("mlProjectLogger")', metadata={'source': 'test_repo\\src\\mlProject\\__init__.py', 'language': <Language.PYTHON: 'python'>}),
 Document(page_content='import os\nimport urllib.request as request\nimport zipfile\nfrom mlProject import logger\nfrom mlProject.utils.common import get_size\nfrom mlProject.entity.config_entity import DataIngestionConfig\nfrom pathlib import Path\n\n\nclass DataIngestion:\n    def __init__(self, config: DataIngestionConfig):\n        self.config = config\n\n    \n    def download_file(self):\n    

##Chunkings

In [9]:

documents_splitter = RecursiveCharacterTextSplitter.from_language(language = Language.PYTHON,
                                                             chunk_size = 2000,
                                                             chunk_overlap = 200)

In [10]:
texts = documents_splitter.split_documents(documents)

In [11]:
len(texts)

19

##Embedding model

In [12]:
os.environ["OPENAI_API_KEY"] = "***************************"

In [13]:
embeddings=OpenAIEmbeddings(disallowed_special=())

###Knowledge base (vector DB)

In [15]:
vectordb = Chroma.from_documents(texts, embedding=embeddings, persist_directory='./data')
vectordb.persist()

ValueError: Expected EmbeddingFunction.__call__ to have the following signature: odict_keys(['self', 'input']), got odict_keys(['self', 'args', 'kwargs'])
Please see https://docs.trychroma.com/embeddings for details of the EmbeddingFunction interface.
Please note the recent change to the EmbeddingFunction interface: https://docs.trychroma.com/migration#migration-to-0416---november-7-2023 


##LLM Wrapper

In [ ]:
# llm = ChatOpenAI(model_name="gpt-4")
llm = ChatOpenAI()

In [ ]:

memory = ConversationSummaryMemory(llm=llm, memory_key = "chat_history", return_messages=True)

In [ ]:
qa = ConversationalRetrievalChain.from_llm(llm, retriever=vectordb.as_retriever(search_type="mmr", search_kwargs={"k":3}), memory=memory)

##QA

In [ ]:
question = "what is DataIngestion class?"

In [ ]:
result = qa(question)
print(result['answer'])